In [537]:
#pliki znajdują się tutaj: https://drive.google.com/drive/folders/1hAwu9fI11cqessZFIVHvILTJGqVXeS0m
#uruchomienie instancji OJS
#PS C:\Users\Cezary> cd D:\IBL\OJS\ojs-3.3.0-16
#PS D:\IBL\OJS\ojs-3.3.0-16> php -S localhost:8000

In [538]:
import pandas as pd
import regex as re
import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, NoAlertPresentException, SessionNotCreatedException, ElementClickInterceptedException, InvalidArgumentException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from tqdm import tqdm

pd.options.display.max_colwidth = 10000

## Przygotowanie funkcji

In [539]:
def jest_autor(x):
    try:
        re.findall('(, \p{Lu})|(^-+|^—+|^–+)', x)[0]
        val = x
    except IndexError:
        val = ''
    return val

def build_author(x):
    try:
        return re.findall('(^.+?)(?=\. \„.+$)', x)[0]
    except IndexError:
        return re.sub('(^.+?)(-|–|—|\p{Ll}\.\)|\p{Ll})(\..+$)', r'\1\2', x)
    
def abstrakt_bio(x):
    x = x.split('\n')
    x = '\n'.join([f'''<span style="font-family: 'Chaparral Pro', serif; color: black;">{e}</span>''' for e in x])
    return x

def biblio_pozycja(x):
    try:
        return re.sub(f"{x['autor pozycji']}.", '', x['bibliografia']).strip()
    except:
        return ''

In [540]:
#data
now = datetime.datetime.now()
year = now.year
month = '{:02d}'.format(now.month)
day = '{:02d}'.format(now.day)

In [541]:
#browser
browser = webdriver.Firefox() 

In [542]:
aktualny_numer = pd.read_excel('OJS_test/FP33-34.xlsx', sheet_name='artykuły po pętli')
aktualny_numer = aktualny_numer[aktualny_numer['język'] == 'pl']
strona_numeru = pd.read_excel('OJS_test/FP33-34.xlsx', sheet_name='strona po pętli')
strona_numeru = strona_numeru[strona_numeru['język'] == 'pl']

In [543]:
aktualny_numer.drop(columns=['bibliografia', 'abstrakt', 'biogram']).head(2)

,lp,autor,ORCID,tytuł artykułu,kategoria,język,tytuł numeru,tag numeru,słowa kluczowe,afiliacja,...,tłumacz,finansowanie,tag autora,link do pdf,link do jpg,pdf,jpg,odnosnik,url_edycji,spis treści
0,1,Marta Rakoczy,0000-0002-7967-2939,"„Najlepsza książka mówiona”. Wat, głos, awangarda i „świadectwa” Europy Środkowo-Wschodniej",Teorie,pl,Forum Poetyki | lato-jesień 2023,lato-jesień 2023,"Aleksander Wat, Czesław Miłosz, pamiętnik mówiony, głos, historia mówiona, gatunki, twórczość słownej, Europa Środkowo-Wschodnia",University of Warsaw,...,NaN,NaN,marta-rakoczy,http://fp.amu.edu.pl/wp-content/uploads/2024/02/MRakoczy_NajlepszaKsiazkaMowiona_ForumPoetyki_33-34_2023.pdf,http://fp.amu.edu.pl/wp-content/uploads/2024/02/MRakoczy_NajlepszaKsiazkaMowiona_ForumPoetyki_33-34_2023-01.jpg,MRakoczy_NajlepszaKsiazkaMowiona_ForumPoetyki_33-34_2023.pdf,MRakoczy_NajlepszaKsiazkaMowiona_ForumPoetyki_33-34_2023-01.jpg,najlepsza-ksiazka-mowiona-wat-glos-awangarda-i-swiadectwa-europy-srodkowo-wschodniej,http://fp.amu.edu.pl/wp-admin/post.php?post=7379&action=edit,"<p style=""text-align: left; margin: 0cm 0cm 15pt; line-height: 15pt; font-size: 11pt; font-family: ChaparralPro-Regular; color: black; letter-spacing: 0.1pt; padding-left: 30px;"" align=""left""><a href=""najlepsza-ksiazka-mowiona-wat-glos-awangarda-i-swiadectwa-europy-srodkowo-wschodniej"">Marta Rakoczy, <em>„Najlepsza książka mówiona”. Wat, głos, awangarda i „świadectwa” Europy Środkowo-Wschodniej</em></a></p>"
2,3,Marta Baron-Milian,0000-0002-5430-4339,Awangarda naprawdę! Biograficzne eksperymenty Sterna z Apollinaire’em,Teorie,pl,Forum Poetyki | lato-jesień 2023,lato-jesień 2023,"awangarda, life writing, biografia, archiwum, Anatol Stern, Guillaume Apollinair\n",University of Silesia,...,NaN,Działania badawcze wsparte ze środków przyznanych w ramach programu Inicjatywa Doskonałości Badawczej Uniwersytetu Śląskiego w Katowicach.,marta-baron-milian,http://fp.amu.edu.pl/wp-content/uploads/2024/02/MBaronMilian_AwangardaNaprawde_ForumPoetyki_33-34_2023.pdf,http://fp.amu.edu.pl/wp-content/uploads/2024/02/MBaronMilian_AwangardaNaprawde_ForumPoetyki_33-34_2023-01.jpg,MBaronMilian_AwangardaNaprawde_ForumPoetyki_33-34_2023.pdf,MBaronMilian_AwangardaNaprawde_ForumPoetyki_33-34_2023-01.jpg,awangarda-naprawde-biograficzne-eksperymenty-sterna-z-apollinaireem,http://fp.amu.edu.pl/wp-admin/post.php?post=7385&action=edit,"<p style=""text-align: left; margin: 0cm 0cm 15pt; line-height: 15pt; font-size: 11pt; font-family: ChaparralPro-Regular; color: black; letter-spacing: 0.1pt; padding-left: 30px;"" align=""left""><a href=""awangarda-naprawde-biograficzne-eksperymenty-sterna-z-apollinaireem"">Marta Baron-Milian, <em>Awangarda naprawdę! Biograficzne eksperymenty Sterna z Apollinaire’em</em></a></p>"


## OJS -- logowanie

In [544]:
browser.get("http://localhost:8000/index.php/test_journal/login")
browser.implicitly_wait(5)
username_input = browser.find_element('id', 'username')
password_input = browser.find_element('id', 'password')

username = 'admin'
password = 'admin'

username_input.clear()
username_input.send_keys(username)
password_input.send_keys(password)

login_button = browser.find_element('css selector', '.submit').click()

## OJS -- dodawanie numeru

In [545]:
browser.get("http://localhost:8000/index.php/test_journal/manageIssues")
create_issue = browser.find_element('css selector', '.pkp_linkaction_icon_add_category')
create_issue.click()

time.sleep(1)
odznacz_tom = browser.find_element('id', 'showVolume').click()
    
wprowadz_numer = browser.find_element('xpath', "//input[@name='number']")
try:
    wprowadz_numer.send_keys('{:.0f}'.format(strona_numeru.at[0, 'numer']))
except ValueError:
    wprowadz_numer.send_keys(strona_numeru.at[0, 'numer'])
rok = browser.find_element('xpath', "//input[@name='year']")
rok.send_keys('{:.0f}'.format(strona_numeru.at[0, 'rok']))
tytul_pl = re.sub('(.+)( \| )(.+)', r'\3', strona_numeru.at[0, 'tytuł numeru']).strip()

try:
    nazwa_numeru = f"No. {'{:.0f}'.format(strona_numeru.at[0, 'numer'])} ({'{:.0f}'.format(strona_numeru.at[0, 'rok'])}): {tytul_pl}"
except ValueError:
    nazwa_numeru = f"No. {strona_numeru.at[0, 'numer']} ({'{:.0f}'.format(strona_numeru.at[0, 'rok'])}): {tytul_pl}"

tytul = browser.find_element('xpath', "//input[@name='title[en_US]']").send_keys(tytul_pl)

wstep_source = browser.find_elements('xpath', "//i[@class='mce-ico mce-i-code']")[0]
wstep_source.click()
wstep_source = browser.find_element('xpath', "//textarea[@class='mce-textbox mce-multiline mce-abs-layout-item mce-first mce-last']")
wstep_source.send_keys(strona_numeru.at[0, 'wstęp'])
wstep_ok = browser.find_element('xpath', "//span[contains(text(),'Ok')]")
wstep_ok.click()

okladka_na_dysku = f"{strona_numeru.at[0, 'folder lokalny']}\{strona_numeru.at[0, 'jpg']}"
przeslij_okladke = browser.find_element('xpath', "//input[@type='file']")
przeslij_okladke.send_keys(okladka_na_dysku)
time.sleep(2)

zapisz_numer = browser.find_element('xpath', "//button[@name='submitFormButton']")
zapisz_numer.click()
print('Done')

Done


## OJS -- dodawanie artykułów

In [546]:
for i, row in tqdm(aktualny_numer.iterrows(), total=aktualny_numer.shape[0]):
    nowe_zgloszenie = browser.get('http://localhost:8000/index.php/test_journal/management/importexport/plugin/QuickSubmitPlugin ')
    dzial_fp = row['kategoria']
    dzial = browser.find_element('xpath', f"//select[@id = 'sectionId']/option[text()='Articles']")
    dzial.click()
    time.sleep(1)
    tytul_art = row['tytuł artykułu'].replace('<i>', '').replace('</i>', '')
    tytul_art_ojs = browser.find_element('xpath', "//input[@name='title[en_US]']")
    tytul_art_ojs.send_keys(tytul_art)
    
    abstrakt_source = browser.find_elements('xpath', "//i[@class='mce-ico mce-i-code']")[0]
    abstrakt_source.click()
    abstrakt_source = browser.find_element('xpath', "//textarea[@class='mce-textbox mce-multiline mce-abs-layout-item mce-first mce-last']")
    abstrakt_source.send_keys(row['abstrakt'])
    abstrakt_ok = browser.find_elements('xpath', "//span[contains(text(),'Ok')]")[-1]
    abstrakt_ok.click()
    
    for s in row['słowa kluczowe'].split(', '):
        s = re.sub(',+', ',', s)
        slowa_kluczowe = browser.find_elements('xpath', "//input[@class='ui-widget-content ui-autocomplete-input']")[0]
        slowa_kluczowe.send_keys(s, Keys.TAB)
    
    if isinstance(row['finansowanie'], str):
        finansowanie = browser.find_elements('xpath', "//input[@class='ui-widget-content ui-autocomplete-input']")[1]
        finansowanie.send_keys(row['finansowanie'])
    
    if len(row['bibliografia']) > 0:
        bibliografia_df = pd.DataFrame(row['bibliografia'].split('\n'), columns=['bibliografia'])
        bibliografia_df = bibliografia_df[bibliografia_df['bibliografia'] != '']       
        
        bibliografia_df['autor pozycji'] = bibliografia_df['bibliografia'].apply(lambda x: build_author(x))
        bibliografia_df['autor pozycji'] = bibliografia_df['autor pozycji'].apply(lambda x: jest_autor(x))
        bibliografia_df['ile małych'] = bibliografia_df['autor pozycji'].apply(lambda x: len(re.findall(' \p{Ll}', x)))
        bibliografia_df['ile słów'] = bibliografia_df['autor pozycji'].apply(lambda x: len(x.split(' ')))
        bibliografia_df['odsetek małych'] = bibliografia_df['ile małych'] / bibliografia_df['ile słów']
        bibliografia_df['autor pozycji'] = bibliografia_df.apply(lambda x: re.sub('(^.+)(\..+$)', r'\1', x['autor pozycji']) if x['odsetek małych'] >= 0.3 else x['autor pozycji'], axis=1)
        
        bibliografia_df['autor pozycji'] = bibliografia_df['autor pozycji'].replace('---|- - - |–––|– – –|———|— — —', np.nan, regex=True).ffill()
        
        bibliografia_df['pozycja'] = bibliografia_df.apply(lambda x: biblio_pozycja(x), axis=1)  
        bibliografia_df['pozycja'] = bibliografia_df.apply(lambda x: x['bibliografia'] if x['pozycja'] == '' else x['pozycja'], axis=1)
        bibliografia_df['pozycja'] = bibliografia_df['pozycja'].str.replace('[- ]{2,}|[— ]{2,}|[— ]{2,}|[– ]{2,}', '', regex=True)
        bibliografia_df['id'] = bibliografia_df.index+1
        bibliografia_df['id'] = bibliografia_df['id']
        bibliografia_df = bibliografia_df.replace(r'^\s*$', np.nan, regex=True)
        bibliografia_df['bibliografia'] =  bibliografia_df[['autor pozycji', 'pozycja']].apply(lambda x: '. '.join(x.dropna().astype(str)).strip(), axis=1)
        bibliografia_df = bibliografia_df['bibliografia']
        bibliografia_df = '\n'.join(bibliografia_df.str.replace('(\. )+', '. ', regex=True).to_list())
        
        bibliografia = browser.find_element('name', 'citationsRaw')
        bibliografia.send_keys(bibliografia_df, Keys.BACK_SPACE)
    
    for a, o, af, b in zip(row['autor'].split('❦'), row['ORCID'].split('❦'), row['afiliacja'].split('❦'), row['biogram'].split('❦')):
        wspolautor_dodaj = browser.find_element('xpath', "//a[contains(text(),'Add Contributor')]")
        wspolautor_dodaj.click()
        autor_imie = re.findall('.+(?= (?!.* ))', a)[0]
        time.sleep(2)
        wprowadz_imie = browser.find_element('name', 'givenName[en_US]')
        wprowadz_imie.send_keys(autor_imie)
        
        autor_nazwisko = re.findall('(?<= (?!.* )).+', a)[0]
        wprowadz_nazwisko = browser.find_element('name', 'familyName[en_US]')
        wprowadz_nazwisko.send_keys(autor_nazwisko)
       
        kontakt = browser.find_element('name', 'email')
        kontakt.send_keys('admin@admin.pl')
        
        kraj = browser.find_element('xpath', "//select[@id = 'country']/option[text()='Poland']").click()
        if len(o) > 0:
            orcid = f"https://orcid.org/{o}"
            wprowadz_orcid = browser.find_element('name', 'orcid').send_keys(orcid)
        
        afiliacja = browser.find_elements('name', "affiliation[en_US]")[-1]
        afiliacja.send_keys(af, Keys.TAB)
        afiliacja.send_keys(Keys.DOWN)
    
        time.sleep(2)
        
        biogram = browser.find_elements('xpath', "//i[@class='mce-ico mce-i-code']")[-1]
        biogram.click()
        biogram = browser.find_element('xpath', "//textarea[@class='mce-textbox mce-multiline mce-abs-layout-item mce-first mce-last']")
        biogram.send_keys(b)
        
        biogram_ok = browser.find_elements('xpath', "//span[contains(text(),'Ok')]")[-1]
        biogram_ok.click()
        time.sleep(2)
    
        kliknij_poza_biogram = browser.find_element('name', 'orcid').click()
        rola_autora = browser.find_element('xpath', "//input[@name='userGroupId' and @value='14']").click()
        zapisz_autora = browser.find_elements('xpath', "//button[@class = 'pkp_button submitFormButton']")[-1].click()
        time.sleep(2)
    
    dodaj_plik = browser.find_element('xpath', "//a[contains(text(),'Add galley')]")
    dodaj_plik.click()
    time.sleep(2)
    etykieta = browser.find_element('xpath', "//input[@class='field text required' and @name = 'label']")
    etykieta.send_keys('PDF')
    jezyk_publikacji = browser.find_element('xpath', "//select[@id = 'galleyLocale']/option[text()='English']")
    jezyk_publikacji.click()
    
    zapisz = browser.find_elements('xpath', "//button[@name='submitFormButton']")
    zapisz[-1].click()
    time.sleep(2)
    
    element_artykulu = browser.find_element('xpath', "//select[@id = 'genreId']/option[text()='Article Text']")
    element_artykulu.click()
    przeslij_pdf = browser.find_element('xpath', "//input[@type='file']")
    przeslij_pdf.send_keys(f"{row['folder lokalny']}\\{row['pdf']}")
    time.sleep(2)
    kontunuuj_button = browser.find_element('id', 'continueButton').click()
    time.sleep(2)
    kontunuuj_button = browser.find_element('id', 'continueButton').click()
    time.sleep(2)
    potwierdz_button = browser.find_element('id', 'continueButton').click()
    time.sleep(2)
    
    while True:
        try:
            zaplanuj_do_publikacji = browser.find_element('id', 'articlePublished').click()
            time.sleep(2)
        except ElementClickInterceptedException:
            potwierdz_button = browser.find_element('id', 'continueButton').click()
            time.sleep(2)
            continue
        break
    
    publikuj_w = browser.find_element('xpath', f"//select[@id = 'issueId']/option[text()='{nazwa_numeru}']")
    publikuj_w.click()
    publikuj_strony = browser.find_element('name', 'pages')
    publikuj_strony.send_keys(row['strony'])
    
    opublikowany_data = browser.find_element('name', 'datePublished-removed')
    opublikowany_data.send_keys(f"{year}-{month}-{day}",Keys.ESCAPE)
    prawa_autorskie = browser.find_element('name', 'copyrightHolder[en_US]')
    prawa_autorskie.send_keys(row['autor'].replace('❦', ', '))
    time.sleep(2)
    rok_praw = browser.find_element('name', 'copyrightYear')
    rok_praw.send_keys(year)        
    
    zapisz = browser.find_elements('xpath', "//button[@class='pkp_button submitFormButton']")
    zapisz[-1].click()
print('Done')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:22<00:00, 32.22s/it]

Done


## OJS -- publikacja numeru

In [547]:
browser.get('http://localhost:8000/index.php/test_journal/manageIssues')
extras = browser.find_element('xpath', "//a[@class='show_extras']")
extras.click()
publish_issue = browser.find_element('xpath', "//a[contains(text(),'Publish Issue')]")
publish_issue.click()
send_email = browser.find_element('xpath', "//input[@id='sendIssueNotification']")
send_email.click()
save_and_edit = browser.find_elements('name', 'submitFormButton')[-1]
save_and_edit.click()
browser.close()
print('Numer opublikowany')

Numer opublikowany


## OJS -- odrzucanie zgłoszeń

In [548]:
browser = webdriver.Firefox()
browser.get("http://localhost:8000/index.php/test_journal/login")
browser.implicitly_wait(5)
username_input = browser.find_element('id', 'username')
password_input = browser.find_element('id', 'password')

username = 'admin'
password = 'admin'

username_input.clear()
username_input.send_keys(username)
password_input.send_keys(password)

login_button = browser.find_element('css selector', '.submit').click()

In [549]:
browser.get('http://localhost:8000/index.php/test_journal/submissions')
submissions = browser.find_elements('xpath', "//a[@class = 'pkpButton']")
submissions = [e for e in submissions if e.get_attribute('href')[-1].isnumeric()]

In [550]:
while submissions:
    submission_urls = []
    for submission in submissions:
        submission_urls.append(submission.get_attribute('href'))
    for url in tqdm(submission_urls):
        if 'wizard/' in url:  
            browser.get(url)
            try:
                checkboxes = browser.find_elements('xpath', "//input[@type = 'checkbox']")
                time.sleep(1)
                for checkbox in checkboxes:
                    checkbox.click()
                save_and_edit = browser.find_elements('name', 'submitFormButton')[-1]
                save_and_edit.click()
                time.sleep(2)
            except: pass
        
            try:
                save_and_edit = browser.find_elements('name', 'submitFormButton')[-1]
                save_and_edit.click()
            except: pass
        
            try:
                time.sleep(2)
                tytul_art = 'test'
                tytul_art_ojs = browser.find_element('xpath', "//input[@name='title[en_US]']")
                tytul_art_ojs.send_keys(tytul_art)
                
                abstrakt_source = browser.find_elements('xpath', "//i[@class='mce-ico mce-i-code']")[-1]
                abstrakt_source.click()
                abstrakt_source = browser.find_element('xpath', "//textarea[@class='mce-textbox mce-multiline mce-abs-layout-item mce-first mce-last']")
                abstrakt_source.send_keys('test')
                abstrakt_ok = browser.find_elements('xpath', "//span[contains(text(),'Ok')]")[-1]
                abstrakt_ok.click()
                
                wspolautor_dodaj = browser.find_element('xpath', "//a[contains(text(),'Add Contributor')]")
                wspolautor_dodaj.click()
                autor_imie = 'test'
                time.sleep(2)
                wprowadz_imie = browser.find_element('name', 'givenName[en_US]')
                wprowadz_imie.send_keys(autor_imie)
                   
                kontakt = browser.find_element('name', 'email')
                kontakt.send_keys('admin@admin.pl')
                
                kraj = browser.find_element('xpath', "//select[@id = 'country']/option[text()='Poland']")
                kraj.click()
                
                rola_autora = browser.find_elements('xpath', "//input[@name='userGroupId' and @value='14']")[-1]
                rola_autora.click()
                zapisz_autora = browser.find_elements('xpath', "//button[@class = 'pkp_button submitFormButton']")[-1].click()
                time.sleep(2)
            except: pass
            
            try:
                zapisz = browser.find_elements('xpath', "//button[@name='submitFormButton']")
                zapisz[-1].click()
                time.sleep(2)
            except: pass
        
            try:
                zapisz = browser.find_elements('xpath', "//button[@name='submitFormButton']")
                zapisz[-1].click()
                time.sleep(2)
            except: pass
        
            try:
                finish_submission = browser.find_element('xpath', "//button[@class='ok pkpModalConfirmButton']")
                finish_submission.click()
                time.sleep(1)
            except: pass
                
        elif 'workflow/' in url:
            try:
                browser.get(url)
                decline = browser.find_element('xpath', "//a[contains(text(),'Decline Submission')]")
                decline.click()
                skip_email = browser.find_element('xpath', "//input[@name='skipEmail' and @value='1']")
                skip_email.click()
                zapisz = browser.find_elements('xpath', "//button[@name='submitFormButton']")
                zapisz[-1].click()
            except NoSuchElementException: pass
    browser.get('http://localhost:8000/index.php/test_journal/submissions')
    submissions = browser.find_elements('xpath', "//a[@class = 'pkpButton']")
    submissions = [e for e in submissions if e.get_attribute('href')[-1].isnumeric()]
print('Done')

Done


## OJS -- usuwanie zgłoszeń

In [551]:
browser.get('http://localhost:8000/index.php/test_journal/submissions#archive')
submissions = browser.find_elements('xpath', "//span[@class = 'fa fa-angle-down']")

In [552]:
for submission in tqdm(submissions):
    try:
        time.sleep(1)
        submission.click()
        delete = browser.find_elements('xpath', "//button[@class='pkpButton pkpButton--isWarnable']")[0]
        delete.click()
        delete_yes = browser.find_elements('xpath', "//button[@class='pkpButton']")[-1]
        delete_yes.click()
    except IndexError: pass
# while submissions:
#     for submission in tqdm(submissions):
#         try:
#             time.sleep(1)
#             submission.click()
#             delete = browser.find_elements('xpath', "//button[@class='pkpButton pkpButton--isWarnable']")[0]
#             delete.click()
#             delete_yes = browser.find_elements('xpath', "//button[@class='pkpButton']")[-1]
#             delete_yes.click()
#         except IndexError: pass
#     browser.get("http://localhost:8000/index.php/test_journal/manageIssues")
#     browser.get('http://localhost:8000/index.php/test_journal/submissions#archive')
#     submissions = browser.find_elements('xpath', "//span[@class = 'fa fa-angle-down']")
print('Done')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:02<00:00,  6.22s/it]

Done


## OJS -- odrzucanie opublikowanych artykułów

In [553]:
browser.get('http://localhost:8000/index.php/test_journal/submissions#archive')
submissions = browser.find_elements('xpath', "//a[@class = 'pkpButton']")
submissions = [e for e in submissions if e.get_attribute('href')[-1].isnumeric()]

In [554]:
while submissions:
    submission_urls = []
    for submission in submissions:
        submission_urls.append(submission.get_attribute('href'))
    for url in tqdm(submission_urls):
        browser.get(url)
        time.sleep(1)
        schedule_for_publication = browser.find_element('xpath', "//button[@class='pkpButton pkpButton--isPrimary']")
        schedule_for_publication.click()
        unpublish = browser.find_element('xpath', "//button[@class='pkpButton pkpButton--isWarnable']")
        unpublish.click()
        unpublish = browser.find_elements('xpath', "//button[@class='pkpButton']")[-1]
        unpublish.click()
        time.sleep(1)
        workflow = browser.find_element('xpath', "//button[@id='workflow-button']")
        workflow.click()
        submission_button = browser.find_element('xpath', "//a[@id='ui-id-1']")
        submission_button.click()
        time.sleep(1)
        assign = browser.find_element('xpath', "//a[@class='pkp_controllers_linkAction pkp_linkaction_requestAccount pkp_linkaction_icon_add_user']")
        assign.click()
        user = browser.find_element('xpath', "//input[@id='user_1']")
        user.click()
        zapisz = browser.find_elements('xpath', "//button[@name='submitFormButton']")
        zapisz[-1].click()
        time.sleep(2)
        change_decision = browser.find_element('xpath', "//button[@class='pkp_workflow_change_decision']")
        change_decision.click()
        
        decline = browser.find_element('xpath', "//a[contains(text(),'Decline Submission')]")
        decline.click()
        skip_email = browser.find_element('xpath', "//input[@name='skipEmail' and @value='1']")
        skip_email.click()
        zapisz = browser.find_elements('xpath', "//button[@name='submitFormButton']")
        zapisz[-1].click()
    browser.get('http://localhost:8000/index.php/test_journal/submissions')
    submissions = browser.find_elements('xpath', "//a[@class = 'pkpButton']")
    submissions = [e for e in submissions if e.get_attribute('href')[-1].isnumeric()]
print('Done')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:29<00:00,  8.92s/it]


Done


## OJS -- usuwanie artykułów

In [555]:
browser.get('http://localhost:8000/index.php/test_journal/submissions#archive')
submissions = browser.find_elements('xpath', "//span[@class = 'fa fa-angle-down']")

In [556]:
for submission in tqdm(submissions):
    try:
        time.sleep(1)
        submission.click()
        delete = browser.find_elements('xpath', "//button[@class='pkpButton pkpButton--isWarnable']")[0]
        delete.click()
        delete_yes = browser.find_elements('xpath', "//button[@class='pkpButton']")[-1]
        delete_yes.click()
    except IndexError: pass
print('Done')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  1.64s/it]

Done


## OJS -- usuwanie numerów

In [557]:
browser.get('http://localhost:8000/index.php/test_journal/manageIssues#back')
extras = browser.find_elements('xpath', "//a[@class='show_extras']")
for extra in tqdm(extras):
    time.sleep(1)
    extra.click()
    delete = browser.find_element('xpath', "//a[contains(text(),'Delete')]")
    time.sleep(1)
    delete.click()
    delete = browser.find_elements('xpath', "//button[@class='ok pkpModalConfirmButton']")[-1]
    time.sleep(1)
    delete.click()
print('Done')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.63s/it]

Done


## Zakończenie pracy

In [558]:
browser.get('http://localhost:8000/index.php/test_journal/manageIssues')

In [559]:
browser.get('http://localhost:8000/index.php/test_journal/submissions')

In [560]:
browser.close()